# Tic-Tac-Toe via Bitboards

This notebook defines the game [tic-tac-toe](https://en.wikipedia.org/wiki/Tic-tac-toe).
It is played on a $3 \times 3$ board.  There are two players, which are called `X`and `O`.  Player `X` starts.  Player `X` always puts an `'X'` into an empty field on the board, while player `O` always puts an `'O'` in an empty field of the board.  The goal of the game for player `X` is to get three **Xs** into a row, column, or diagonal line, while player `O` needs to get three **Os** into a row, column, or diagonal line.

In this notebook, the board is represented as a *bitboard*, i.e. the states are represented as integers.  If `s` is a state, the first nine bits of `s` specify the positions of the `X`es, while the second nine bits specify the positions of the `O`s.  Player `X` is encoded as the number `0`,  while player `O` is encoded as the number `1`. 

from Stoetmann

Jupyter Notebook had issues using the libary we used to read inputs. Therefor this notebook is only for documentation and performing Minimax

``players`` and ``start`` define the available players and which one starts

In [1]:
let players = [0, 1];
let start = 0;

The function ``setBits`` takes a list of ``Integers`` and then set the repective bits inside an integer

In [2]:
function setBits(bits) {
    let result = 0;
    bits.forEach(bit => {
        result = result | 1 << bit;
    });
    return result;
};

In [3]:
//Test
setBits([1,3,5])
//Expected: 42

42

``AllLines`` defines all possible winning conditions for a player. As longs as one is fulfilled the game is won

In [4]:
let AllLines = [  
                setBits([0,1,2]), // 1st row
                setBits([3,4,5]), // 2nd row
                setBits([6,7,8]), // 3rd row
                setBits([0,3,6]), // 1st column
                setBits([1,4,7]), // 2nd column
                setBits([2,5,8]), // 3rd column
                setBits([0,4,8]), // falling diagonal
                setBits([2,4,6]) // rising diagonal
               ];

The ``setBit`` is simelar to the ``setBits`` function but only accepts one integer and therfore only sets one Bit

In [5]:
function setBit(bit) {
    return 1 << bit;
};

In [6]:
//Test
setBit(3)
//Expected: 8

8

``toBoard`` prints the current state of the board

In [7]:
function toBoard(state) {
    let result = "+-+-+-+\n";
    for (let i = 0; i < 9; i++) {
        if ((state & (1 << i)) !== 0) {
            result += "|X";
        }
        else if ((state & (1 << (i + 9))) !== 0) {
            result += '|O';
        }
        else {
            result += '| ';
        }
        if (((i + 1) % 3) == 0) {
            result += '|\n+-+-+-+\r\n';
        }
    }
    return result;
};

In [8]:
//Test
console.log(toBoard(setBits([0,1,5,8,9+3,9+4,9+7])))

+-+-+-+
|X|X| |
+-+-+-+
|O|O|X|
+-+-+-+
| |O|X|
+-+-+-+



``allFree`` gives for a ``state`` all free fields as a list of their index

In [9]:
function allFree(state) {
    let freeCells = [];
    for (let i = 0; i < 9; i++) {
        if ((state & ( (1 << i) + (1 <<(i + 9)) )) === 0) {
            freeCells.push(i);
        }
    }
    return freeCells;
};

In [10]:
allFree(setBits([0,1,5,8,9+3,9+4,9+7]))

[ 2, 6 ]

``nextStates`` gives for a ``state`` and a ``player`` all states that could occur next

In [11]:
function nextStates(state, player) {
    const empty = allFree(state);
    let result = [];
    empty.forEach(i => {
        let nextState = state | setBit((player * 9) + i);
        result.push(nextState);
    });
    return result;
};

In [12]:
let state = setBits([2,3,5,10,13,15]);
console.log("current state: \n" + toBoard(state));

console.log("nextStates: \n");
nextStates(state, 0).forEach(i => {
    console.log(toBoard(i));
});

current state: 
+-+-+-+
| |O|X|
+-+-+-+
|X|O|X|
+-+-+-+
|O| | |
+-+-+-+

nextStates: 

+-+-+-+
|X|O|X|
+-+-+-+
|X|O|X|
+-+-+-+
|O| | |
+-+-+-+

+-+-+-+
| |O|X|
+-+-+-+
|X|O|X|
+-+-+-+
|O|X| |
+-+-+-+

+-+-+-+
| |O|X|
+-+-+-+
|X|O|X|
+-+-+-+
|O| |X|
+-+-+-+



The ``utility`` function returns if a given ``player`` has won or lost in a specific ``state``.

A ``1`` is retured for a win of player, a ``-1`` for a loss. ``0`` means its a draw and for ``null`` its not yet finished

In [13]:
function utility(state, player) {
    for (let i = 0; i < AllLines.length; i++) {
        const mask = AllLines[i];
        if ((state & mask) == mask) {
            return 1 - (2 * player);
        }
        if (((state >> 9) & mask) == mask) {
            return -1 + (2 * player);
        }
    };
    if (((state & 511) | (state >> 9)) !== 511) {
        return null;
    }
    return 0;
};

In [14]:
s1 = setBits([0, 2, 5, 6, 7, 1+9, 3+9, 4+9, 8+9]);
console.log(toBoard(s1));
console.log(utility(s1, 0));

+-+-+-+
|X|O|X|
+-+-+-+
|O|O|X|
+-+-+-+
|X|X|O|
+-+-+-+

0


The function heuristic tries to guess the value of a state.  As it is never called in terminal states, it assumes that the game will be drawn.

from Stoetmann

In [15]:
function heuristic(state, player) {
    return 0;
};

``finished`` checks if for a given ``state`` the game is finished. This is true if ``utility`` of a player return ``1``,``-1`` or ``0``

In [16]:
function finished(state) {
    return (utility(state, 0) !== null);
};

In [17]:
let s = setBits([0, 2, 5, 6, 7, 1+9, 3+9, 4+9, 8+9]);
console.log(toBoard(s));
console.log(finished(s));

+-+-+-+
|X|O|X|
+-+-+-+
|O|O|X|
+-+-+-+
|X|X|O|
+-+-+-+

true


Everything below is just for documentation purposes ``readline-sync`` did only work in a regular install of ``node-js`` without jupyter notebook

In [1]:
const readline = require('readline-sync');

``getMove`` allows the player to input a move and return the new state

In [19]:
function getMove(state) {
    let move = readline.question("Enter move here: ");
    move = move.split(",");
    let row = parseInt(move[0]);
    let col = parseInt(move[1]);
    let mask = setBit(9 + row * 3 + col);
    if ((state & mask) === 0) {
        return state | mask;
    }
    else {
        console.log("Illegal input. Please try again.")
        return getMove(state);
    }
};

In [20]:
//console.log(getMove(0));

``finalMsg`` prints a message if the player has won

In [21]:
function finalMsg(state) {
    if (finished(state) === true) {
        if (utility(state, 1) === 1) {
            console.log("You have won!");
        }
        else if (utility(state, 1) === -1){
            console.log("You have lost!");
        }
        else {
            console.log("It's a draw.");
        }
        return true;
    };
    return false;
};

In [22]:
finalMsg(setBits([0, 2, 3, 6, 1+9,  4+9, 5+9]));

You have lost!


true